In [102]:
from pymongo import MongoClient
from pyspark.sql import SparkSession
import matplotlib.pyplot as plt
from pyspark.sql.functions import col, array_contains


In [103]:
# Connect to MongoDB
client = MongoClient('mongodb://mongodb:27017/')
db = client.Youtube_Database
collection = db.youtube_data_us

# Print the first document
print(collection.find_one())

# Close the connection
client.close()

{'_id': ObjectId('676ebf9db2412e2d5055f480'), 'video_id': '3C66w5Z0ixs', 'title': 'I ASKED HER TO BE MY GIRLFRIEND...', 'publishedAt': '2020-08-11T19:20:14Z', 'channelId': 'UCvtRTOMP2TqYqu51xNrqAzg', 'channelTitle': 'Brawadis', 'categoryId': 22, 'trending_date': '2020-08-12T00:00:00Z', 'tags': ['brawadis', 'prank', 'basketball', 'skits', 'ghost', 'funny videos', 'vlog', 'vlogging', 'NBA', 'browadis', 'challenges', 'bmw i8', 'faze rug', 'faze rug brother', 'mama rug and papa rug'], 'view_count': 1514614, 'likes': 156908, 'dislikes': 5855, 'comment_count': 35313, 'thumbnail_link': 'https://i.ytimg.com/vi/3C66w5Z0ixs/default.jpg', 'comments_disabled': False, 'ratings_disabled': False, 'description': 'SUBSCRIBE to BRAWADIS ▶ http://bit.ly/SubscribeToBrawadis\r\rFOLLOW ME ON SOCIAL\r▶ Twitter: https://twitter.com/Brawadis\r▶ Instagram: https://www.instagram.com/brawadis/\r▶ Snapchat: brawadis\r\rHi! I’m Brandon Awadis and I like to make dope vlogs, pranks, reactions, challenges and basketba

In [108]:
spark = SparkSession.builder \
    .appName("youtube_us") \
    .config("spark.mongodb.input.uri", "mongodb://mongodb:27017/Youtube_Database") \
    .config("spark.mongodb.output.uri", "mongodb://mongodb:27017/Youtube_Database") \
    .config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:3.0.1") \
    .getOrCreate()


In [109]:
df_us = spark.read.format("mongo").option("uri", "mongodb://mongodb:27017/Youtube_Database.youtube_data_us").load()
df_br = spark.read.format("mongo").option("uri", "mongodb://mongodb:27017/Youtube_Database.youtube_data_br").load()
df_ca = spark.read.format("mongo").option("uri", "mongodb://mongodb:27017/Youtube_Database.youtube_data_ca").load()
df_de = spark.read.format("mongo").option("uri", "mongodb://mongodb:27017/Youtube_Database.youtube_data_de").load()
df_fr = spark.read.format("mongo").option("uri", "mongodb://mongodb:27017/Youtube_Database.youtube_data_fr").load()
df_gb = spark.read.format("mongo").option("uri", "mongodb://mongodb:27017/Youtube_Database.youtube_data_gb").load()
df_in = spark.read.format("mongo").option("uri", "mongodb://mongodb:27017/Youtube_Database.youtube_data_in").load()
df_jp = spark.read.format("mongo").option("uri", "mongodb://mongodb:27017/Youtube_Database.youtube_data_jp").load()
df_kr = spark.read.format("mongo").option("uri", "mongodb://mongodb:27017/Youtube_Database.youtube_data_kr").load()
df_mx = spark.read.format("mongo").option("uri", "mongodb://mongodb:27017/Youtube_Database.youtube_data_mx").load()
df_ru = spark.read.format("mongo").option("uri", "mongodb://mongodb:27017/Youtube_Database.youtube_data_ru").load()

In [117]:
df_all = df_us.union(df_br).union(df_ca).union(df_de).union(df_fr).union(df_gb).union(df_in).union(df_jp).union(df_kr).union(df_mx).union(df_ru)


In [118]:
df_all.printSchema()

root
 |-- _id: struct (nullable = true)
 |    |-- oid: string (nullable = true)
 |-- categoryId: integer (nullable = true)
 |-- channelId: string (nullable = true)
 |-- channelTitle: string (nullable = true)
 |-- comment_count: integer (nullable = true)
 |-- comments_disabled: boolean (nullable = true)
 |-- description: string (nullable = true)
 |-- dislikes: integer (nullable = true)
 |-- likes: integer (nullable = true)
 |-- publishedAt: string (nullable = true)
 |-- ratings_disabled: boolean (nullable = true)
 |-- tags: string (nullable = true)
 |-- thumbnail_link: string (nullable = true)
 |-- title: string (nullable = true)
 |-- trending_date: string (nullable = true)
 |-- video_id: string (nullable = true)
 |-- view_count: integer (nullable = true)



In [119]:
filtered_df_nvidia = df_all.filter(
    (col('description').contains('nvidia')) |
    (col('title').contains('nvidia')) |
    (col('tags').contains('nvidia')) |  
    (col('channelTitle').contains('nvidia')) |
    (col('thumbnail_link').contains('nvidia'))
)

num_rows = filtered_df_nvidia.count()
print(f"Número de linhas que contêm 'nvidia': {num_rows}")

Número de linhas que contêm 'nvidia': 2568


In [122]:
filtered_df_ibm = df_all.filter(
    (col('description').contains('ibm')) |
    (col('title').contains('ibm')) |
    (col('tags').contains('ibm')) |  
    (col('channelTitle').contains('ibm')) |
    (col('thumbnail_link').contains('ibm'))
)

num_rows = filtered_df_ibm.count()
print(f"Número de linhas que contêm 'ibm': {num_rows}")

Número de linhas que contêm 'ibm': 958


In [123]:
filtered_df_dell = df_all.filter(
    (col('description').contains('dell')) |
    (col('title').contains('dell')) |
    (col('tags').contains('dell')) |  
    (col('channelTitle').contains('dell')) |
    (col('thumbnail_link').contains('dell'))
)

num_rows = filtered_df_dell.count()
print(f"Número de linhas que contêm 'dell': {num_rows}")

Número de linhas que contêm 'dell': 958


In [124]:
filtered_df_intel = df_all.filter(
    (col('description').contains('intel')) |
    (col('title').contains('intel')) |
    (col('tags').contains('intel')) |  
    (col('channelTitle').contains('intel')) |
    (col('thumbnail_link').contains('intel'))
)

num_rows = filtered_df_intel.count()
print(f"Número de linhas que contêm 'intel': {num_rows}")

Número de linhas que contêm 'intel': 17326


In [125]:
filtered_df_microsoft = df_all.filter(
    (col('description').contains('microsoft')) |
    (col('title').contains('microsoft')) |
    (col('tags').contains('microsoft')) |  
    (col('channelTitle').contains('microsoft')) |
    (col('thumbnail_link').contains('microsoft'))
)

num_rows = filtered_df_microsoft.count()
print(f"Número de linhas que contêm 'microsoft': {num_rows}")

Número de linhas que contêm 'microsoft': 2958


In [126]:
filtered_df_sony = df_all.filter(
    (col('description').contains('sony')) |
    (col('title').contains('sony')) |
    (col('tags').contains('sony')) |  
    (col('channelTitle').contains('sony')) |
    (col('thumbnail_link').contains('sony'))
)

num_rows = filtered_df_sony.count()
print(f"Número de linhas que contêm 'sony': {num_rows}")

Número de linhas que contêm 'sony': 17574
